In [26]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017, username='root', password='example')

db = client['quiz']

users_collecition = db['users']

users_collecition.drop()

In [27]:
from itertools import islice
import json
from datetime import datetime

with open ('data.jsonl', 'r') as f:
    for line in islice(f, 400):
        data = json.loads(line)
        data['post']['timestamp'] = datetime.fromisoformat(data['post']['timestamp'])
        for comment in data['comments']:
            comment['comment']['timestamp'] = datetime.fromisoformat(comment['comment']['timestamp'])
        
        users_collecition.insert_one(data)


{'_id': ObjectId('65f211f49e6d60fc0ea2e2b5'), 'user': 'Mrs. Christine Sawyer', 'post_length': 232}


1. Operaciones CRUD básicas (Crear, Leer, Actualizar, Eliminar)

In [3]:
# 1. Insertar 5 nuevos documentos de usuario con datos únicos.
doc1 = {"user": "Jerson", "user_age": 21, "user_email": "jerson@gmail.com"}
doc2 = {"user": "Carlos", "user_age": 22, "user_email": "carlos@gmail.com"}
doc3 = {"user": "Luis", "user_age": 23, "user_email": "luis@gmail.com"}
doc4 = {"user": "Pedro", "user_age": 24, "user_email": "pedro@gmail.com"}
doc5 = {"user": "Maria", "user_age": 35, "user_email": "maria@gmail.com"}

# insertar documentos a la colección
users_collecition.insert_one(doc1)
users_collecition.insert_one(doc2)
users_collecition.insert_one(doc3)
users_collecition.insert_one(doc4)
users_collecition.insert_one(doc5)

InsertOneResult(ObjectId('65f13685444a74168b3a70da'), acknowledged=True)

In [ ]:
# 2. Buscar todos los usuarios con edad mayor a 25.
list(db.users.find({"user_age": {"$gt": 25}}, {"_id": 0, "user": 1, "user_age": 1, "user_email": 1}))

In [13]:
# 3. Buscar un usuario por dirección de correo electrónico.
email_to_find = "jerson@gmail.com"

query = {"user_email": email_to_find}
result = users_collecition.find_one(query)
if result:
    print("Usuario encontrado:")
    print(result)
else:
    print("No se encontró ningún usuario con ese correo electrónico.")


Usuario encontrado:
{'_id': ObjectId('65f1098b4da40f1600c11cd2'), 'user': 'Jerson', 'user_age': 21, 'user_email': 'jerson@gmail.com'}


In [ ]:
# 4. Actualizar el contenido de la publicación de un usuario específico.

In [ ]:
# 5. Eliminar un documento de usuario y todos sus comentarios asociados.

2. Consultas con operadores

In [ ]:
# 1. Buscar todos los usuarios cuyo nombre de usuario contenga la palabra S̈mith.̈ (Expresiones regulares)
list(db.users.find({"user": {"$regex": ".*Smith"}}, {"_id": 0, "user": 1, "user_age": 1, "user_email": 1}))

In [ ]:
# 2. Buscar publicaciones creadas después de una fecha específica (por ejemplo, 2000-01-01).
from datetime import datetime
fecha_especifica = datetime(2014, 1, 1)
query = {"post.timestamp": {"$gt": fecha_especifica}}
result = users_collecition.find(query)

print("Publicaciones creadas después de", fecha_especifica)
for post in result:
    print(post)


In [ ]:
# 3. Buscar usuarios que aún no han publicado ningún comentario

In [ ]:
# 4. Encontrar la edad promedio de todos los usuarios.
pipeline = [
    {
        "$group": {
            "_id": None,
            "total_users": {"$sum": 1},
            "total_age": {"$sum": "$user_age"}
        }
    },
    {
        "$project": {
            "_id": 0,
            "average_age": {"$divide": ["$total_age", "$total_users"]}
        }
    }
]

result = users_collecition.aggregate(pipeline)

average_age = list(result)[0]["average_age"]
print("Edad promedio de todos los usuarios:", average_age)


Edad promedio de todos los usuarios: 28.1


In [ ]:
# 5. Buscar documentos donde la longitud del contenido de la publicación sea superior a 100 caracteres

3. Operadores lógicos y proyección

In [ ]:
# 1. Buscar usuarios que tengan más de 20 años pero menos de 30. (Combinar operadores)
list(db.users.find({"user_age": {"$gt": 20, "$lt": 30}}, {"_id": 0, "user": 1, "user_age": 1, "user_email": 1}))

In [ ]:
# 2. Recuperar solo el nombre de usuario y la edad del usuario de todos los documentos. (Proyección)

In [ ]:
# 3. Buscar todas las publicaciones y excluir el campo ”comentarios”

In [ ]:
# 4. Buscar usuarios con al menos un comentario en su matriz ”comentarios”

In [ ]:
# 5. Buscar comentarios publicados por usuarios mayores de 40 años. (Consultas anidadas)
query = {
    "comments": {
        "$elemMatch": {
            "user_age": {"$gt": 40}
        }
    }
}

result = users_collecition.find(query)

# Iterar sobre los resultados
print("Comentarios de usuarios mayores de 40 años:")
for user in result:
    for comment in user['comments']:
        if comment['user_age'] > 40:
            print(comment)


4. Ordenar y limitar

In [55]:
# 1. Ordenar todos los usuarios por edad en orden descendente.
cursor = db.users.aggregate([
    {"$sort": {"user_age": -1}},
    {"$project": {"_id": 0, "user": 1, "user_age": 1, "user_email": 1}}
    ])

list(cursor)

[{'user': 'Margaret Flores',
  'user_email': 'stephensonjohn@example.net',
  'user_age': 43},
 {'user': 'Margaret Flores',
  'user_email': 'stephensonjohn@example.net',
  'user_age': 43},
 {'user': 'Margaret Flores',
  'user_email': 'stephensonjohn@example.net',
  'user_age': 43},
 {'user': 'Margaret Flores',
  'user_email': 'stephensonjohn@example.net',
  'user_age': 43},
 {'user': 'Margaret Flores',
  'user_email': 'stephensonjohn@example.net',
  'user_age': 43},
 {'user': 'Margaret Flores',
  'user_email': 'stephensonjohn@example.net',
  'user_age': 43},
 {'user': 'Margaret Flores',
  'user_email': 'stephensonjohn@example.net',
  'user_age': 43},
 {'user': 'Todd Park', 'user_email': 'megan76@example.com', 'user_age': 42},
 {'user': 'Todd Park', 'user_email': 'megan76@example.com', 'user_age': 42},
 {'user': 'Todd Park', 'user_email': 'megan76@example.com', 'user_age': 42},
 {'user': 'Todd Park', 'user_email': 'megan76@example.com', 'user_age': 42},
 {'user': 'Todd Park', 'user_email

In [ ]:
# 2. Buscar los 3 usuarios principales con las publicaciones más recientes (en función de la marca de tiempo).

In [3]:
# 3. Omitir los primeros 5 resultados y recuperar los próximos 10 usuarios.
result = users_collecition.find().skip(5).limit(10)
print("Próximos 10 usuarios después de omitir los primeros 5:")
for user in result:
    print(user)


Próximos 10 usuarios después de omitir los primeros 5:
{'_id': ObjectId('65f211f49e6d60fc0ea2e2a8'), 'user': 'Joann Bowen', 'user_email': 'tmartin@example.org', 'user_age': 19, 'post': {'title': 'Random title for Joann Bowen', 'content': 'Random content for Joann Bowen Mind the form local easy evidence. Get leave media good. Contain sort heavy like amount. Point arrive nature modern rock me imagine as.', 'timestamp': datetime.datetime(2015, 5, 5, 15, 6, 49, 482000)}, 'comments': [{'user': 'Lucas Rogers', 'user_email': 'kmorrison@example.com', 'user_age': 32, 'comment': {'text': 'Random text Trouble prove kind begin film perhaps line. His miss sit through whether society right decide. Serve difficult cover feeling.', 'timestamp': datetime.datetime(1974, 9, 16, 12, 59, 52, 349000)}}, {'user': 'Timothy Gallegos', 'user_email': 'joseph02@example.com', 'user_age': 28, 'comment': {'text': 'Random text Treat tend live. Through successful far black.\nLater field war president enter commercial.

5. Marco de agregación

In [52]:
# 1. Utilizar el pipeline de agregación para calcular el número total de usuarios
cursor = db.users.aggregate([
    {"$group": {"_id": "$user"}}
])

lista_cursor = list(cursor)

print(f"La cantidad total de usuarios es {len(lista_cursor)}")

La cantidad total de usuarios es 9


In [8]:
# 2. Encontrar al usuario con el contenido de publicación más largo (en función del número de caracteres).
result = users_collecition.aggregate([
    {"$project": {
        "user": 1,
        "post_length": {"$strLenCP": "$post.content"}
    }},
    {"$sort": {"post_length": -1}},
    {"$limit": 1}
])

for user in result:
    print(user)


{'_id': ObjectId('65f211f49e6d60fc0ea2e2b5'), 'user': 'Mrs. Christine Sawyer', 'post_length': 232}


In [ ]:
# 3. Agrupar a los usuarios por edad y contar la cantidad de usuarios en cada grupo de edad.

In [ ]:
# 4. Calcular la longitud promedio de los comentarios en todos los comentarios.

In [ ]:
# 5. Cambiar la forma como se presentan los datos, de tal forma que cada usuario tenga la lista de todos
# los posts asociados a él. No se deben agregar el detalle de los comentarios, solo un nuevo campo con
# el total de comentarios.

6. Actualización y eliminación con operadores

In [54]:
# 1. Incrementar la edad de todos los usuarios en 1.
db.users.update_many({}, {"$inc": {"user_age": 1}})

UpdateResult({'n': 400, 'nModified': 400, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [ ]:
# 2. Establecer el campo ”contenido” de una publicación específica en una cadena vacía

In [ ]:
# 3. Eliminar todos los comentarios de la publicación de un usuario
usuario = "Joann Bowen"  
titulo_publicacion = "Random title for Joann Bowen"  
query = {
    "user": usuario, 
    "post.title": titulo_publicacion  
}

update_query = {
    "$set": {
        "comments": []  
    }
}
result = users_collecition.update_many(query, update_query)
print("Número de comentarios eliminados:", result.modified_count)


In [ ]:
# 4. Eliminar todos los usuarios con direcciones de correo electrónico que terminen en ”@example.org”.
query = {
    "user_email": {"$regex": r"@example\.org$"} 
}

result = users_collecition.delete_many(query)
print("Número de usuarios eliminados:", result.deleted_count)
                                                                                    #Elimina terminados en @example.org, no @example.com ni @example.net


7. Indexación y optimización avanzadas

In [59]:
# 1. Crear un índice en el campo ”user_email” para búsquedas de correo electrónico más rápidas.
db.users.create_index([("user_email", 1)])

'user_email_1'

In [ ]:
# 2. Explicar el plan de consulta para encontrar usuarios con edad mayor a 30



#
#La consulta para encontrar usuarios con edad mayor a 30 en sintaxis de MongoDB puede verse de la siguiente manera:
#                                                   query = {"user_age": {"$gt": 30}}
#


#Plan de consulta:
#Cuando ejecutamos esta consulta MongoDB utilizará el índice creado en el campo "user_age" para optimizar la búsqueda, lo que mejora su rendimiento y permite obtener resultados de forma eficiente
#   MongoDB buscará en el índice de user_age los valores que son mayores a 30.
#   Después usará estos valores del índice para acceder directamente a los documentos que satisfacen la condición de la consulta.
#   Finalmente, MongoDB devolverá los documentos encontrados que cumplan con el criterio de búsqueda.



In [ ]:
#3. Discutir las ventajas y desventajas de almacenar comentarios dentro del documento de usuario.
